# Instructions
1) Download webpage (HTML only)
2) Rename HTML file to ``input.htm``
3) Insert your ChatGPT API key
4) Run "home page praser" in the following, will output 
    - Method: Due to variations in page structures, it is easier to chunk and parse through HTML file into CSV file
    - In: ``input.htm``. Out: ``1homepage.csv``
5) Run ``obtain about page``
    - In: ``1homepage.csv``. Out: ``2aboutpage.csv``
6) Manually clean up formatting and potential artifacts
    - This can be due to ad-hoc instances where canidates is still being enstated or remove thus no URL sor artifact from chunking the HTML file for processing
7) Run ``obtain wikipedia link via Google``
    - In: ``2aboutpage.csv``. Out: ``3aboutpage-n-wiki.csv``
8) Run ``wiki vcard scraper`` to see where they went to university via wikipedia
    - In: ``3aboutpage-n-wiki.csv``. Out: ``4aboutpage-n-wiki-vcard.csv``
    - Common issues: If incorrect ans is returned, likely due to wrong wikipedia link or couldn't find exact potlicain name. University on wikipedia vcard can also be wrapped weirdly in CSS, thus may need to manually eneter university name
9) Run ``Clean wikipedia vcard university into individual columns``
    - In: ``4aboutpage-n-wiki-vcard.csv``. Out: ``5aboutpage-n-cleaned-uni.csv``
10) Manully clean up formatting and artifacts under "office-start-date" column. 
    - Anomolies, formatting issues or x2 dates (instead of x1 date) is likely the wrong wikipedia page found.
10) Run ``scrape about page bio``
    - In: ``5aboutpage-n-cleaned-uni.csv``. Out: ``6scraped-bio-n-cleaned-uni.csv``
11) Run ``check if uni is prestige & if bio mentioned``
    - In: ``6scraped-bio-n-cleaned-uni.csv``. Out: ``7bio-checked-n-uni-checked.csv``
    - This also requires ``T25-list.csv``, listing top 25 universities in general and liberal arts category from US News. Alternate names and syntax of unis are also added here, e.g. "University of California, Berkeley", "University of California - Berkeley", "Cal", "UCB", "Berkeley".
    - This will check if university is elite & extract the sentences that mentioned "academy", "university", "college", "school" in ``uni_in_bio`` column. This is to aid manual vetting and fuzzy logic matching after. 
12) Run ``check for university aliases``
    - In: ``7bio-checked-n-uni-checked.csv``. Out: ``8final_university_matching.csv``
    - This uses fuzzymatching to check if university is mentioned.
13) Run ``LLM to vet uni mention in bio``
    - In:``8final_university_matching.csv``. Out: ``9LLM-vetted.csv``
    - Using deepseek + fuzzy matching to check if uni has been mentioned. 
14) Check "LLM_vetted_result" for discrepencies. 1 is discrepency and may require further manual revision by looking at ``uni_in_bio`` column. 

In [ ]:
! python -m venv bio-scrape
! .\bio-scrape\Scripts\activate
! pip install jupyter
! python -m ipykernel install --user --name=bio-scrape --display-name "Python (bio-scrape)"


In [ ]:
%pip install pandas requests openai beautifulsoup4 aiohttp



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.9.0-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
     ---------------------------------------- 0.0/64.7 kB ? eta -:--:--
     ------------------------- -------------- 41.0/64.7 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 64.7/64.7 kB 1.8 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/599.1 kB ? eta -:--:--
   -- ------------------------------------- 41.0/599.1 kB ? eta -:--:--
   -- ------------------------------------- 41.0/599.1 kB ? eta -:--:--
   ----------- ---------------------------- 174.1/599.1 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 358.4/599.1 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 599.1/599.1 kB 2.7 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:

import pandas as pd                   
import requests                       
import json                           
import re
import openai                             
import asyncio                        
from bs4 import BeautifulSoup       
import csv
import os 
import time
import urllib.parse
from urllib.parse import urljoin
from fuzzywuzzy import fuzz

In [ ]:
openai.api_key = "OPENAI_API_KEY"

## Obtain home page

In [ ]:
def call_chatgpt(prompt: str) -> str:
    """Call OpenAI's ChatGPT API to extract data from HTML."""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts data from HTML tables."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=4096,
            temperature=0
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return ""

def split_html(content: str, chunk_size: int = 10000):
    """Split HTML content into manageable chunks."""
    return [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]

def extract_data_from_html(html_chunks):
    """Extract structured data from HTML chunks."""
    data = []
    for chunk in html_chunks:
        prompt = f"Extract a list of politicians from the following HTML chunk, including their state, district, name, party, and URL, into a CSV. \n\n{chunk}"
        result = call_chatgpt(prompt)
        if result:
            rows = result.strip().split("\n")
            for row in rows:
                if row:
                    data.append(row.split(","))
    return data

def save_to_csv(data, output_file):
    """Save extracted data to a CSV file."""
    headers = ["State", "District", "Name", "URL", "Party"]
    try:
        with open(output_file, mode='w', newline='', encoding='UTF-16') as file:
            writer = csv.writer(file)
            writer.writerow(headers)
            writer.writerows(data)
        print(f"Data saved to {output_file}")
    except Exception as e:
        print(f"Error saving to CSV: {e}")

def main(html_file: str, output_file: str):
    """Main function to process an HTML file and save extracted data."""
    try:
        with open(html_file, 'r', encoding='UTF-16') as file:
            content = file.read()
        html_chunks = split_html(content)
        extracted_data = extract_data_from_html(html_chunks)
        save_to_csv(extracted_data, output_file)
    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    main('input.htm', '1homepage.csv')

Error processing file: [Errno 2] No such file or directory: 'input.htm'


## Obtain about page

In [ ]:
# Keywords to search for in links
keywords = ["about", "biography", "bio", "full biography", "background", "profile", "who-is", "meet-"]

# Function to find about-related URLs from a given URL
def find_about_page(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        filtered_links = []
        for link in soup.find_all('a', href=True):
            href = link.get('href').lower()
            text = link.get_text().lower()

            if any(keyword in href or keyword in text for keyword in keywords):
                # Convert relative URLs to absolute URLs
                full_url = urljoin(url, href)
                filtered_links.append(full_url)

        # Return the first valid link found, or None if not found
        return filtered_links[0] if filtered_links else None

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

# Read URLs from input CSV file
input_file = '1homepage.csv'
output_file = '2aboutpage.csv'

with open(input_file, 'r', encoding='UTF-16') as infile, open(output_file, 'w', newline='', encoding='UTF-16') as outfile:
    reader = csv.DictReader(infile)
    
    if 'Homepage' not in reader.fieldnames:
        raise ValueError("Input CSV file must have a column named 'Homepage'")
    
    fieldnames = reader.fieldnames + ['candidates official page URL']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        url = row['Homepage'].strip()
        
        if not url.startswith("http"):
            url = "http://" + url  # Ensure URL is properly formatted

        about_url = find_about_page(url)
        row['about-page'] = about_url if about_url else "Not Found"
        
        writer.writerow(row)
        print(f"Processed: {url} -> {about_url if about_url else 'Not Found'}")


Processed: https://barrymoore.house.gov -> https://barrymoore.house.gov/about
Processed: https://figures.house.gov/ -> https://figures.house.gov/about
Processed: https://mikerogers.house.gov/ -> https://mikerogers.house.gov/about/
Processed: https://aderholt.house.gov/ -> https://aderholt.house.gov/about-robert
Processed: https://strong.house.gov -> https://strong.house.gov/about
Processed: https://palmer.house.gov/ -> https://palmer.house.gov/about/full-biography
Processed: https://sewell.house.gov/ -> https://sewell.house.gov/meet-terri
Processed: https://begich.house.gov/ -> https://begich.house.gov/about
Processed: https://radewagen.house.gov -> https://radewagen.house.gov/about
Processed: https://schweikert.house.gov/ -> https://schweikert.house.gov/about/
Processed: https://crane.house.gov -> https://crane.house.gov/about/
Processed: https://ansari.house.gov/ -> https://ansari.house.gov/about
Processed: https://stanton.house.gov/ -> https://stanton.house.gov/about
Processed: http

## Obtain wikipedia link via Google

In [ ]:
def find_wikipedia_url(candidate_name, canidate_state, about_page_url=None):
    """
    Search Wikipedia for a politician and return the most likely Wikipedia page URL.
    If a valid about_page_url is provided and contains a Wikipedia link, it is returned directly.
    """
    if about_page_url and 'wikipedia.org/wiki/' in about_page_url:
        return about_page_url
    # Prepare the search query
    search_query = "\"" + candidate_name + "\"" + canidate_state + " politician"
    encoded_query = urllib.parse.quote(search_query)
    
    # Try the Google search method first
    try:
        google_search_url = f"https://www.google.com/search?q={encoded_query}+site:wikipedia.org"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        
        response = requests.get(google_search_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the first Wikipedia link in Google results
        for link in soup.find_all('a', href=True):
            href = link['href']
            if 'wikipedia.org/wiki/' in href:
                start = href.find('url=') + 4 if 'url=' in href else 0
                end = href.find('&', start) if '&' in href[start:] else len(href)
                wikipedia_url = href[start:end]
                
                if 'wikipedia.org/wiki/' in wikipedia_url:
                    return wikipedia_url.split('&')[0]
    except:
        pass
    
    # If Google method fails, try Wikipedia's API directly
    try:
        api_url = "https://en.wikipedia.org/w/api.php"
        params = {
            'action': 'query',
            'list': 'search',
            'srsearch': f"{candidate_name} {canidate_state} politician senate house", # <-- change search query here
            'format': 'json'
        }
        
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data['query']['search']:
            page_title = data['query']['search'][0]['title']
            wikipedia_url = f"https://en.wikipedia.org/wiki/{urllib.parse.quote(page_title.replace(' ', '_'))}"
            return wikipedia_url
    except:
        pass
    
    return "Not Found"

def process_candidates(input_file, output_file):
    """
    Process the input CSV file and add Wikipedia URLs.
    """
    # Read the input CSV
    df = pd.read_csv(input_file)
    
    # Add a new column for Wikipedia URLs
    df['wikipedia_url'] = ""
    
    # Process each row
    for index, row in df.iterrows():
        candidate_name = row["Name"] if "Name" in df.columns else ""
        canidate_state = row["State"] if "State" in df.columns else ""
        about_page = row['about-page'] if 'about-page' in df.columns else ""
        
        print(f"Processing {index + 1}/{len(df)}: {candidate_name} {canidate_state}")
        
        wikipedia_url = find_wikipedia_url(candidate_name, canidate_state, about_page)
        df.at[index, 'wikipedia_url'] = wikipedia_url
        
        time.sleep(1)
    
    # Save to new CSV file
    df.to_csv(output_file, index=False)
    print(f"Processing complete. Results saved to {output_file}")

if __name__ == "__main__":
    input_filename = "2aboutpage.csv"
    output_filename = "3aboutpage-n-wiki.csv"
    
    process_candidates(input_filename, output_filename)


Processing 1/441: Moore, Barry Alabama
Processing 2/441: Figures, Shomari Alabama
Processing 3/441: Rogers, Mike Alabama
Processing 4/441: Aderholt, Robert Alabama
Processing 5/441: Strong, Dale Alabama
Processing 6/441: Palmer, Gary Alabama
Processing 7/441: Sewell, Terri Alabama
Processing 8/441: Begich, Nicholas Alaska
Processing 9/441: Radewagen, Aumua Amata American Samoa
Processing 10/441: Schweikert, David Arizona
Processing 11/441: Crane, Elijah Arizona
Processing 12/441: Ansari, Yassamin Arizona
Processing 13/441: Stanton, Greg Arizona
Processing 14/441: Biggs, Andy Arizona
Processing 15/441: Ciscomani, Juan Arizona
Processing 16/441: Grijalva, Raúl  Arizona
Processing 17/441: Hamadeh, Abraham Arizona
Processing 18/441: Gosar, Paul Arizona
Processing 19/441: Crawford, Eric Arkansas
Processing 20/441: Hill, J. Arkansas
Processing 21/441: Womack, Steve Arkansas
Processing 22/441: Westerman, Bruce Arkansas
Processing 23/441: LaMalfa, Doug California
Processing 24/441: Huffman, Ja

## Wiki vcard scraper

In [3]:
def get_info_from_wikipedia(url):
    """
    Scrapes a Wikipedia URL's infobox for Education/Alma Mater and
    Assumed Office/In Office dates.

    Args:
        url (str): The Wikipedia URL to scrape.

    Returns:
        tuple: A tuple containing (education, office_date).
               Returns specific error messages or defaults if info not found.
    """
    # Handle potential non-string or NaN URLs
    if not isinstance(url, str) or pd.isna(url) or not url.startswith('http'):
        return "Invalid or Missing URL", "Invalid or Missing URL"
    try:
        headers = {'User-Agent': 'MyCoolScraper/1.0 (contact@example.com)'} # Be polite, identify your bot
        response = requests.get(url, headers=headers, timeout=10) # Add timeout
        response.raise_for_status() # Checks for HTTP errors (4xx or 5xx)

        soup = BeautifulSoup(response.text, 'html.parser')
        infobox = soup.find('table', class_='infobox') # More general infobox class

        # Try another common infobox class if the first fails
        if not infobox:
                infobox = soup.find('table', class_='infobox vcard')

        if not infobox:
            return "Infobox Not Found", "Infobox Not Found"

        education = "Education Not Found in Infobox"
        office_date = "Office Date Not Found in Infobox" # Default for office date

        # Search for relevant information within the infobox rows
        rows = infobox.find_all('tr')
        for i, row in enumerate(rows):
            header = row.find('th')
            data_cell = row.find('td')

            if header:
                header_text = header.text.strip().lower()

                # Find Education or Alma Mater
                if 'education' in header_text or 'alma mater' in header_text:
                    if data_cell:
                        # Improve extraction: get text, handle line breaks, strip whitespace
                        education = data_cell.get_text(separator='; ', strip=True)
                        # Don't break yet, continue searching for office date

                # Find Assumed Office or In Office (prioritize Assumed Office if both exist)
                if 'assumed office' in header_text:
                   # The date is often in the next row's td if the header spans the full row,
                   # or in the same row's td if it's a label-data pair.
                   # Check if the header has colspan="2" or if there's no td in the current row
                   if header.get('colspan') == '2' or not data_cell:
                       # Look in the next row's td
                       if i + 1 < len(rows):
                           next_row_data = rows[i+1].find('td')
                           if next_row_data:
                               # Check if the next row cell is not intended for something else (like predecessor/successor)
                               prev_th = rows[i+1].find('th')
                               if not prev_th or not prev_th.text.strip(): # Ensure it's not a labeled row
                                    office_date = next_row_data.get_text(separator=' ', strip=True)
                                    # Prefer 'Assumed office' if found, potentially overwrite 'In office'
                   elif data_cell: # Standard th/td row structure
                       office_date = data_cell.get_text(separator=' ', strip=True)

                elif 'in office' in header_text and office_date == "Office Date Not Found in Infobox":
                    # Similar logic as 'Assumed office', but only if 'Assumed office' wasn't found
                   if header.get('colspan') == '2' or not data_cell:
                       if i + 1 < len(rows):
                           next_row_data = rows[i+1].find('td')
                           if next_row_data:
                               prev_th = rows[i+1].find('th')
                               if not prev_th or not prev_th.text.strip():
                                   office_date = next_row_data.get_text(separator=' ', strip=True)
                   elif data_cell:
                       office_date = data_cell.get_text(separator=' ', strip=True)

            # Sometimes the "Assumed office" text is not in a `th` but directly above the date `td`
            # Example: <tr><td colspan="2" style="border-bottom:none"><span class="nowrap"><b>Assumed office</b></span> <br>January 3, 2021</td></tr>
            elif data_cell and data_cell.get('colspan') == '2':
                 bold_span = data_cell.find('span', class_='nowrap')
                 if bold_span and bold_span.find('b'):
                     bold_text = bold_span.find('b').text.strip().lower()
                     if bold_text == 'assumed office':
                         # Extract text following the <br> tag if present, otherwise the whole cell text after the span
                         br = data_cell.find('br')
                         if br and br.next_sibling:
                             office_date = br.next_sibling.strip()
                         else: # Fallback if no <br> or text after it
                             office_date = data_cell.get_text(separator=' ', strip=True).replace(bold_span.text.strip(),'').strip()

                     elif bold_text == 'in office' and office_date == "Office Date Not Found in Infobox":
                         br = data_cell.find('br')
                         if br and br.next_sibling:
                              office_date = br.next_sibling.strip()
                         else:
                             office_date = data_cell.get_text(separator=' ', strip=True).replace(bold_span.text.strip(),'').strip()

        # Clean up potential "[...] years ago)" text from dates like 'Incumbent' might add
        if '(' in office_date and 'age' not in office_date.lower(): # Avoid removing age parens
             office_date = office_date.split('(')[0].strip()

        return education, office_date

    except requests.exceptions.RequestException as e:
        return f"Request Error: {e}", f"Request Error: {e}" # Specific error for network issues
    except Exception as e:
        # Log the URL that caused the parsing error for debugging
        print(f"Parsing Error for URL: {url} - Error: {e}")
        return f"Parsing Error: {e}", f"Parsing Error: {e}" # Specific error for parsing issues

# --- Main Script ---

# Load CSV file
csv_file = "3bio-checked-n-uni-checked-senate.csv"  # Update with your actual file name
try:
    # Try common encodings if ISO-8859-1 fails
    try:
        df = pd.read_csv(csv_file, encoding="UTF-8")
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file, encoding="ISO-8859-1") # Try UTF-16 as a common alternative

except FileNotFoundError:
    raise Exception(f"Error: CSV file '{csv_file}' not found. Cannot proceed without the CSV file.")
except Exception as e:
    raise Exception(f"Error reading CSV file '{csv_file}': {e}. Cannot proceed.")

# Ensure the URL column exists
if 'wikipedia_url' not in df.columns:
    print(f"Error: Column 'wikipedia_url' not found in '{csv_file}'.")
    exit() # Exit if the column is missing

# Apply the function and store results
education_results = []
office_date_results = []
total_rows = len(df['wikipedia_url'])

print(f"Starting Wikipedia scrape for {total_rows} URLs...")

for index, url in enumerate(df['wikipedia_url']):
    print(f"Processing URL {index + 1}/{total_rows}: {url}") # Progress indicator
    edu_info, office_info = get_info_from_wikipedia(url)
    education_results.append(edu_info)
    office_date_results.append(office_info)
    time.sleep(0.5) # Wait half a second between requests - BE POLITE!

print("Scraping complete. Adding results to DataFrame...")

# Add the results as new columns to the DataFrame
df['education'] = education_results
df['office_start_date'] = office_date_results # New column for the date

# Save the results to a new CSV
output_csv = "4aboutpage-n-wiki-vcard.csv" # Use a new output name
df.to_csv(output_csv, index=False, encoding='UTF-8') # Save as UTF-16 for better compatibility

print(f"Extraction complete. Results saved to {output_csv}")

Starting Wikipedia scrape for 100 URLs...
Processing URL 1/100: https://en.wikipedia.org/wiki/Todd_Young
Processing URL 2/100: https://en.wikipedia.org/wiki/Ron_Wyden
Processing URL 3/100: https://en.wikipedia.org/wiki/Roger_Wicker
Processing URL 4/100: https://en.wikipedia.org/wiki/Sheldon_Whitehouse
Processing URL 5/100: https://en.wikipedia.org/wiki/Peter_Welch
Processing URL 6/100: https://en.wikipedia.org/wiki/Elizabeth_Warren
Processing URL 7/100: https://en.wikipedia.org/wiki/Raphael_Warnock
Processing URL 8/100: https://en.wikipedia.org/wiki/Mark_Warner
Processing URL 9/100: https://en.wikipedia.org/wiki/Chris_Van_Hollen
Processing URL 10/100: https://en.wikipedia.org/wiki/Tommy_Tuberville
Processing URL 11/100: https://en.wikipedia.org/wiki/Thom_Tillis
Processing URL 12/100: https://en.wikipedia.org/wiki/John_Thune
Processing URL 13/100: https://en.wikipedia.org/wiki/Dan_Sullivan_(U.S._senator)
Processing URL 14/100: https://en.wikipedia.org/wiki/Tina_Smith
Processing URL 15/1

## Clean wikipedia vcard university into individual columns

In [5]:
# Load the CSV file
df = pd.read_csv('4aboutpage-n-wiki-vcard.csv')

def clean_and_split_education(education_str):
    if not isinstance(education_str, str):
        return []  # Return an empty list when the input is not a string
    cleaned_str = re.sub(r'\([^)]*\)', '', education_str)
    universities = [uni.strip() for uni in cleaned_str.split(';') if uni.strip()]
    return universities

df['education'] = df['education'].apply(clean_and_split_education)

max_universities = df['education'].apply(len).max()

for i in range(max_universities):
    df[f'University_{i+1}'] = df['education'].apply(lambda x: x[i] if i < len(x) else '')

df.drop(columns=['education'], inplace=True)

df.to_csv('5aboutpage-n-cleaned-uni.csv', index=False)

print("Cleaning and splitting complete.")

Cleaning and splitting complete.


## Scrape about page bio
- First following block contains openAI code and subsequent block contains deepseek code

In [ ]:
# OpenAI code
client = openai.OpenAI()  # Create the client

def scrape_bio(url):
    if not url or url.lower() == "not found":
        print(f"Skipping invalid URL: {url}")
        return ""

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {str(e)}")
        return ""

    soup = BeautifulSoup(response.text, 'html.parser')
    page_text = soup.get_text(separator="\n")

    chunks = [page_text[i:i + 2000] for i in range(0, len(page_text), 2000)]
    full_bio = ""

    for chunk in chunks:
        prompt = f"Extract the following text exactly as it is without summarizing:\n\n{chunk}"
        try:
            chat_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a text extraction tool."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1500,
                temperature=0.0,
            )
            full_bio += chat_response.choices[0].message.content.strip() + "\n"
        except openai.OpenAIError as e:
            print(f"OpenAI API error for {url}: {str(e)}")
            return ""

    return full_bio.strip()

# File paths
input_file = "5aboutpage-n-cleaned-uni.csv"
output_file = "6scraped-bio-n-cleaned-uni.csv"

# Read input CSV and process each URL
with open(input_file, 'r', encoding='UTF-16') as csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = reader.fieldnames + ['bio']

    with open(output_file, 'w', newline='', encoding='UTF-16') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            url = row.get('candidates official page URL', '').strip()
            print(f"Processing URL: {url}")

            if url:
                row['bio'] = scrape_bio(url)
            else:
                row['bio'] = ""

            writer.writerow(row)

print(f"Scraping completed. Results saved to {output_file}")


Processing URL: https://barrymoore.house.gov/about
OpenAI API error for https://barrymoore.house.gov/about: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Processing URL: https://figures.house.gov/about
OpenAI API error for https://figures.house.gov/about: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Processing URL: https://mikerogers.house.gov/about/
OpenAI API error for https://mikerogers.house.gov/about/: Error code: 429 - {'error': {'message': 'You exceeded your

In [ ]:
import os
import csv
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import time # Added for potential rate limiting

# --- Environment Variable Loading ---
# Load environment variables from .env file located in the script's directory or parent directories
load_dotenv()

# Get the API key using its variable NAME defined in the .env file
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# CRITICAL: Check if the API key was loaded successfully
if not DEEPSEEK_API_KEY:
    raise ValueError("DeepSeek API key not found in environment variables. "
                     "Ensure you have a .env file in the script's directory "
                     "with the line: DEEPSEEK_API_KEY=your_actual_key")

# --- DeepSeek API Interaction ---
def query_deepseek(prompt):
    """Send a prompt to DeepSeek's API and return the response."""
    headers = {
        # Use the correctly loaded API key here
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "deepseek-chat",  # Verify this is the correct/desired model
        "messages": [
            {"role": "system", "content": "You are a text extraction tool. Extract the following text exactly as it is, without summarizing or altering it."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 8192, # Increased slightly, adjust as needed based on chunk size and typical response
        "temperature": 0.0 # Keep temperature low for exact extraction
    }

    try:
        response = requests.post(
            "https://api.deepseek.com/chat/completions", # Verify the correct API endpoint
            headers=headers,
            json=data,
            timeout=30 # Added a longer timeout for potentially slow API responses
        )
        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        # Print detailed error, including status code if available
        status_code = e.response.status_code if e.response is not None else "N/A"
        print(f"API request failed: {str(e)} (Status Code: {status_code})")
        # Optional: Add a small delay before retrying or failing
        # time.sleep(1)
        return None

# --- Web Scraping Function ---
def scrape_bio(url):
    """Scrapes text from a URL and uses DeepSeek API for extraction."""
    if not url or url.lower() == "not found":
        print(f"Skipping invalid or missing URL: {url}")
        return ""

    print(f"Processing URL: {url}")
    try:
        # Added headers to potentially mimic a browser
        scrape_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, timeout=15, headers=scrape_headers) # Increased timeout
        response.raise_for_status() # Check for HTTP errors like 404 Not Found
        response.encoding = response.apparent_encoding # Try to handle encoding better

    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {str(e)}")
        return "" # Return empty string on fetch error

    soup = BeautifulSoup(response.text, 'html.parser')

    # Try to find main content area if possible (improves quality, requires site inspection)
    # Example: main_content = soup.find('main') or soup.find('article') or soup.find(id='content')
    # if main_content:
    #     page_text = main_content.get_text(separator="\n", strip=True)
    # else:
    #     page_text = soup.get_text(separator="\n", strip=True) # Fallback to full text

    page_text = soup.get_text(separator="\n", strip=True) # Using full text as per original code

    if not page_text:
        print(f"No text extracted from {url}")
        return ""

    # Define chunk size - ensure it's less than API token limits considering prompt+response
    chunk_size = 1800 # Characters per chunk (adjust based on testing and token limits)
    chunks = [page_text[i:i + chunk_size] for i in range(0, len(page_text), chunk_size)]
    full_bio = ""
    print(f"Splitting text from {url} into {len(chunks)} chunks.")

    for i, chunk in enumerate(chunks):
        print(f"  Processing chunk {i+1}/{len(chunks)}...")
        # The prompt asks the model to just return the text provided
        prompt = f"Extract the following text exactly as it is without summarizing or adding anything:\n\n---\n{chunk}\n---"
        api_response = query_deepseek(prompt)

        if api_response and "choices" in api_response and len(api_response["choices"]) > 0:
            extracted_text = api_response["choices"][0]["message"]["content"].strip()
            full_bio += extracted_text + "\n" # Append extracted text
            # Optional: Add a small delay between API calls to avoid rate limits
            # time.sleep(0.5)
        else:
            print(f"Failed to get valid response for chunk {i+1} from {url}")
            # Decide how to handle partial failure: continue, return partial, or return ""
            # Returning "" on first failure to ensure data integrity, modify if partial results are okay
            return "" # Failed to process a chunk, abort for this URL

    print(f"Successfully processed {url}")
    return full_bio.strip()

# --- Main Execution Logic ---
# Define input and output file paths
input_file = "5aboutpage-n-cleaned-uni.csv"
output_file = "6scraped-bio-n-cleaned-uni.csv"

try:
    # Read input CSV and process each URL
    with open(input_file, 'r', encoding='UTF-16') as csvfile:
        # Check if file is empty
        first_char = csvfile.read(1)
        if not first_char:
            print(f"Error: Input file '{input_file}' is empty.")
            exit()
        csvfile.seek(0) # Reset file pointer

        reader = csv.DictReader(csvfile)

        # Ensure the 'candidates official page URL' column exists
        if 'candidates official page URL' not in reader.fieldnames:
             raise ValueError(f"Column 'candidates official page URL' not found in '{input_file}'. "
                              f"Available columns: {reader.fieldnames}")

        # Prepare fieldnames for the output file
        fieldnames = reader.fieldnames + ['bio']

        # Write to output CSV
        with open(output_file, 'w', newline='', encoding='UTF-16') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                # Get the URL, default to empty string if column missing or empty
                url = row.get('candidates official page URL', '').strip()

                # Scrape the biography using the function
                row['bio'] = scrape_bio(url) # scrape_bio handles empty/invalid URLs

                # Write the original row data plus the new bio
                writer.writerow(row)

    print("-" * 20)
    print(f"Scraping completed. Results saved to {output_file}")
    print("-" * 20)

except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Processing URL: https://barrymoore.house.gov/about
API request failed: 401 Client Error: Unauthorized for url: https://api.deepseek.com/chat/completions
Failed to get valid response for https://barrymoore.house.gov/about
Processing URL: https://figures.house.gov/about
API request failed: 401 Client Error: Unauthorized for url: https://api.deepseek.com/chat/completions
Failed to get valid response for https://figures.house.gov/about
Processing URL: https://mikerogers.house.gov/about/
API request failed: 401 Client Error: Unauthorized for url: https://api.deepseek.com/chat/completions
Failed to get valid response for https://mikerogers.house.gov/about/
Scraping completed. Results saved to scraped-bio-n-cleaned-uni.csv


## Check if uni is prestige & if bio mentioned

In [13]:
df = pd.read_csv('6scraped-bio-n-cleaned-uni.csv', encoding='latin1')
elite_df = pd.read_csv('T25-list.csv')
elite_list = elite_df['University'].tolist()

def is_elite(university):
    return 1 if university in elite_list else 0

# Add elite status for each university column
for i in range(1, 6):
    df[f'University_{i}_Elite'] = df[f'University_{i}'].apply(is_elite)

def combined_uni_in_bio(row):
    # Extract university mentions only once
    bio_text = row['bio']
    if not isinstance(bio_text, str):
        return ""
    
    keywords = ["college", "school", "university", "academy", "degree", "masters", "bachelor", "associate", "phd", "doctorate", "GED", "MBA", "cum laude", "attended", "graduated", "studied", "enrolled", "graduate", "alumni", "alma mater"]
    sentences = re.split(r'(?<=[.!?])\s+|\n+', bio_text)
    matches = [
        sentence.strip()
        for sentence in sentences
        if any(keyword in sentence.lower() for keyword in keywords)
           and "military academy nominations" not in sentence.lower()
    ]
    
    return "\n".join(matches) if matches else ""

# Create one combined column with all university-related sentences
df['uni_in_bio'] = df.apply(combined_uni_in_bio, axis=1)

df.to_csv('7bio-checked-n-uni-checked.csv', index=False)
print("Elite status and combined uni mention checks added.")


Elite status and combined uni mention checks added.


# Check for university aliases

In [14]:
def clean_text(text):
    if pd.isna(text):
        return ""
    # Fix encoding issues
    text = text.replace('ÃƒÂƒÃ‚Â¢ÃƒÂ‚Ã‚Â€ÃƒÂ‚Ã‚Â™', "'")
    text = text.replace('ÃƒÂ‚Ã‚Â', "'")
    # Remove punctuation and make lowercase
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    return ' '.join(text.split())  # Remove extra whitespace

def is_university_in_bio(university, bio):
    if pd.isna(university) or pd.isna(bio):
        return 0
    
    clean_uni = clean_text(university)
    clean_bio = clean_text(bio)
    
    # Dictionary of university variations (key = main name, value = list of aliases)
    university_variations = {
        'adelphi university': ['adelphi'],
        'air university': ['au'],
        'albright college': ['albright'],
        'alliant international university': ['alliant'],
        'alma college': ['alma'],
        'american university': ['au'],
        'amherst college': ['amherst'],
        'appalachian state university': ['app state', 'asu'],
        'arizona state university tempe': ['arizona state university', 'asu', 'arizona state'],
        'arizona state university phoenix': ['asu phoenix', 'arizona state phoenix'],
        'auburn university': ['auburn'],
        'ball state university': ['ball state'],
        'baylor university': ['baylor'],
        'biola university': ['biola'],
        'binghamton university': ['suny binghamton', 'binghamton'],
        'birmingham southern college': ['birminghamsouthern college', 'bsc'],
        'boston college': ['bc'],
        'boston university': ['bu'],
        'brigham young university': ['byu'],
        'brooklyn college': ['brooklyn'],
        'brown university': ['brown'],
        'butler community college': ['butler cc'],
        'california institute of technology': ['caltech'],
        'california polytechnic state university san luis obispo': ['cal poly'],
        'capella university': ['capella'],
        'carnegie mellon university': ['cmu'],
        'case western reserve university': ['case western', 'cwru'],
        'charleston southern university': ['charleston southern'],
        'chattanooga state community college': ['chattanooga state', 'chattanooga cc'],
        'clark county community college': ['clark county cc'],
        'college of the holy cross': ['holy cross'],
        'columbia university': ['columbia'],
        'columbus state university': ['columbus state'],
        'concordia college': ['concordia'],
        'copiaha lincoln community college': ['copiaha lincoln cc'],
        'cornell university': ['cornell'],
        'dallas theological seminary': ['dallas seminary'],
        'dartmouth college': ['dartmouth'],
        'deep springs college': ['deep springs'],
        'duke university': ['duke'],
        'embry-riddle aeronautical university': ['erau', 'embry riddle'],
        'emory university': ['emory'],
        'fairleigh dickinson university': ['fdu', 'fairleigh dickinson'],
        'florida state university': ['fsu'],
        'fort hays state university': ['fhsu', 'fort hays state'],
        'george mason university': ['gmu'],
        'george washington university': ['gwu', 'gw'],
        'georgetown university': ['georgetown'],
        'georgia institute of technology': ['georgia tech', 'gatech'],
        'gonzaga university': ['gonzaga'],
        'grace college and seminary': ['grace college'],
        'harvard university': ['harvard law school', 'harvard college', 'harvard'],
        'indiana university bloomington': ['indiana university', 'iu bloomington', 'iu'],
        'indiana university indianapolis': ['iupui', 'iu indianapolis'],
        'iowa state university': ['isu', 'iowa state'],
        'johns hopkins university': ['jhu', 'johns hopkins'],
        'kansas state university': ['k-state', 'ksu'],
        'kent state university': ['kent state'],
        'london school of economics': ['lse'],
        'louisiana state university': ['lsu'],
        'magdalen college oxford': ['magdalen college'],
        'marshall university': ['marshall'],
        'massachusetts institute of technology': ['mit'],
        'miami university': ['miami ohio', 'miami u'],
        'michigan state university': ['msu'],
        'mississippi state university': ['mississippi state', 'msu'],
        'missouri valley college': ['missouri valley'],
        'montana state university': ['montana state', 'msu'],
        'morehouse college': ['morehouse'],
        'naval postgraduate school': ['nps'],
        'new mexico highlands university': ['nmhu'],
        'new york university': ['nyu'],
        'north carolina state university': ['nc state', 'ncsu'],
        'northeastern university': ['northeastern'],
        'northern illinois university': ['niu'],
        'northwestern university': ['northwestern'],
        'ohio state university': ['osu', 'ohio state'],
        'oklahoma state university institute of technology': ['osuit'],
        'pennsylvania state university': ['penn state', 'psu'],
        'pennsylvania state university university park': ['penn state main campus'],
        'pomona college': ['pomona'],
        'presbyterian college': ['presbyterian'],
        'princeton university': ['princeton'],
        'purdue university': ['purdue'],
        'queens college oxford': ["queen's college, oxford", "queen's college"],
        'rutgers university': ['rutgers'],
        'rutgers university new brunswick': ['rutgers main campus'],
        'saint louis university': ['slu'],
        'samford university': ['cumberland school of law', 'samford'],
        'san diego state university': ['sdsu'],
        'shippensburg university': ['shippensburg'],
        'smith college': ['smith'],
        'south dakota state university': ['sdsu'],
        'southern college of optometry': ['sco'],
        'southern methodist university': ['smu'],
        'southern state college': ['southern state'],
        'southwestern baptist theological seminary': ['swbts'],
        'st lawrence university': ['saint lawrence university', 'st lawrence'],
        'st marys university texas': ["st. mary's university, texas", "saint mary's university"],
        'stanford university': ['stanford'],
        'stetson university': ['stetson'],
        'stony brook university': ['suny stony brook'],
        'swarthmore college': ['swarthmore'],
        'texas a&m university': ['tamu', 'texas am'],
        'texas tech university': ['ttu', 'texas tech'],
        'trinity college cambridge': ['trinity college'],
        'trinity university': ['trinity'],
        'truman state university': ['truman state'],
        'tufts university': ['tufts'],
        'tulane university': ['tulane'],
        'union theological seminary': ['union seminary'],
        'united states merchant marine academy': ['usmma', 'merchant marine academy'],
        'united states military academy': ['west point', 'usma'],
        'united states naval academy': ['annapolis', 'usna'],
        'university of alabama': ['ua', 'alabama university', 'bama'],
        'university of arizona': ['uofa', 'u of a'],
        'university of arkansas': ['uark', 'arkansas'],
        'university of california berkeley': ['uc berkeley', 'cal', 'ucb', 'university of california, berkeley'],
        'university of california los angeles': ['ucla', 'university of california, los angeles'],
        'university of california santa barbara': ['ucsb', 'university of california, santa barbara'],
        'university of chicago': ['uchicago'],
        'university of colorado boulder': ['cu boulder', 'colorado'],
        'university of connecticut': ['uconn'],
        'university of dayton': ['ud', 'dayton'],
        'university of delaware': ['udel', 'delaware'],
        'university of detroit': ['detroit mercy', 'udm'],
        'university of florida': ['uf', 'florida'],
        'university of georgia': ['uga'],
        'university of guam': ['guam university', 'uog'],
        'university of hawaii': ['uh', 'hawaii'],
        'university of houston': ['uh'],
        'university of idaho': ['uidaho'],
        'university of illinois at urbana-champaign': ['uiuc', 'illinois'],
        'university of iowa': ['uiowa', 'iowa'],
        'university of kansas': ['ku', 'kansas'],
        'university of kentucky': ['uk', 'kentucky'],
        'university of london': ['uol'],
        'university of louisville': ['uofl', 'louisville'],
        'university of maryland baltimore': ['umd baltimore', 'university of maryland, baltimore'],
        'university of maryland college park': ['umd', 'maryland'],
        'university of maryland university college': ['umuc', 'umgc'],
        'university of mary': ['umary'],
        'university of michigan': ['umich', 'michigan'],
        'university of minnesota': ['u of m', 'umn'],
        'university of mississippi': ['ole miss'],
        'university of missouri': ['mizzou', 'mu'],
        'university of missouri kansas city': ['umkc', 'university of missouri–kansas city'],
        'university of nebraska lincoln': ['unl', 'university of nebraska, lincoln'],
        'university of nevada reno': ['unr', 'university of nevada, reno'],
        'university of new mexico': ['unm'],
        'university of north carolina at chapel hill': ['unc', 'north carolina'],
        'university of northern iowa': ['uni'],
        'university of notre dame': ['notre dame'],
        'university of oklahoma': ['ou', 'oklahoma'],
        'university of oregon': ['uoregon', 'uo'],
        'university of pennsylvania': ['upenn', 'penn'],
        'university of south carolina': ['usc', 'south carolina'],
        'university of south dakota': ['usd'],
        'university of southern california': ['usc'],
        'university of southern mississippi': ['southern miss', 'usm'],
        'university of tennessee': ['ut', 'utk'],
        'university of texas at austin': ['ut austin', 'utexas', 'ut'],
        'university of virginia': ['uva'],
        'university of washington': ['uw', 'u dub'],
        'university of wisconsin milwaukee': ['uw', 'uwm', 'university of wisconsin, milwaukee'],
        'university of wyoming': ['uwyo'],
        'vanderbilt university': ['vanderbilt', 'vandy'],
        'villanova university': ['villanova', 'nova'],
        'virginia tech': ['virginia polytechnic institute', 'vt'],
        'wake forest university': ['wake', 'wake forest'],
        'washington state university': ['wsu', 'wash state'],
        'washington university': ['washu'],
        'wayne state university': ['wsu', 'wayne state'],
        'wesleyan university': ['wesleyan'],
        'willamette university': ['willamette'],
        'williams college': ['williams'],
        'yale university': ['yale'],
        'oxford university': ['university of oxford', 'oxford', 'pembroke college', 'merton college', 'st hildas college oxford'],
        'cambridge university': ['university of cambridge', 'cambridge', 'king\'s college', 'st johns college', 'saint johns college cambridge']
    }
    
    # Check direct match first
    if clean_uni in clean_bio:
        return 1
    
    # Check if any variation matches
    for main_name, aliases in university_variations.items():
        if clean_uni in main_name:  # Check if our university is a main name
            for alias in aliases + [main_name]:
                if alias in clean_bio:
                    return 1
        elif any(alias in clean_uni for alias in aliases):  # Check if our university is an alias
            if main_name in clean_bio:
                return 1
    
    # Special handling for law/medical schools
    if 'law school' in clean_bio and ('law' in clean_uni or 'university' in clean_uni):
        return 1
    
    # Try fuzzy matching as last resort
    if fuzz.partial_ratio(clean_uni, clean_bio) >= 85:
        return 1
    
    return 0

# Read the CSV file
df = pd.read_csv('7bio-checked-n-uni-checked.csv', encoding='latin1')

# Create new columns for each university column, returning 1 for True and 0 for False
for i in range(1, 6):
    col_name = f'University_{i}'
    new_col_name = f'University_{i}_in_bio'
    df[new_col_name] = df.apply(lambda row: is_university_in_bio(row[col_name], row['uni_in_bio']), axis=1)

# Save the results
df.to_csv('8final_university_matching.csv', index=False, encoding='utf-8')
print("Final matching completed. Results saved to '8final_university_matching.csv'")

Final matching completed. Results saved to '8final_university_matching.csv'


# LLM to vet uni mention in bio

In [ ]:
import pandas as pd
import requests
import json

# Load the CSV file
df = pd.read_csv('8final_university_matchingsenate.csv', encoding='Latin1')

# Initialize DeepSeek API parameters
DEEPSEEK_API_KEY = 'DEEPSEEK-API-KEY'  # Replace with your actual API key
DEEPSEEK_API_URL = 'https://api.deepseek.com/v1/chat/completions'  # Example endpoint, adjust as needed

def check_university_in_bio(university, bio_text):
    """Use DeepSeek API to check if university is mentioned in bio"""
    if pd.isna(university) or pd.isna(bio_text):
        return 0
    
    prompt = f"Does the following text mention '{university}'? Answer only '1' for yes or '0' for no. Text: {bio_text}"
    
    headers = {
        'Authorization': f'Bearer {DEEPSEEK_API_KEY}',
        'Content-Type': 'application/json'
    }
    
    payload = {
        'model': 'deepseek-chat',  # Adjust model as needed
        'messages': [
            {'role': 'user', 'content': prompt}
        ],
        'max_tokens': 1
    }
    
    try:
        response = requests.post(DEEPSEEK_API_URL, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        answer = result['choices'][0]['message']['content'].strip()
        return int(answer) if answer in ['0', '1'] else 0
    except Exception as e:
        print(f"Error checking university {university}: {e}")
        return 0

# Create new columns for DeepSeek verification
for i in range(1, 6):
    col_name = f'University_{i}'
    new_col_name = f'DeepSeek_vetted_uni{i}'
    
    df[new_col_name] = df.apply(
        lambda row: check_university_in_bio(row[col_name], row['uni_in_bio']),
        axis=1
    )
# Save the updated dataframe
df.to_csv('9LLM-vetted.csv', index=False)

# Add column based on comparing the university_in_bio and DeepSeek_vetted values
df["LLM_vetted_result"] = df.apply(
    lambda row: 0 if (
        row["University_1_in_bio"] == row["DeepSeek_vetted_uni1"] and
        row["University_2_in_bio"] == row["DeepSeek_vetted_uni2"] and
        row["University_3_in_bio"] == row["DeepSeek_vetted_uni3"] and
        row["University_4_in_bio"] == row["DeepSeek_vetted_uni4"] and
        row["University_5_in_bio"] == row["DeepSeek_vetted_uni5"]
    ) else 1,
    axis=1
)
# Save the updated dataframe
df.to_csv('9LLM-vetted.csv', index=False)

print("University verification complete. Results saved to 9LLM-vetted.csv")

University verification complete. Results saved to 9LLM-vetted.csv
